# GermEval -- TF.IDF


The original dataset is an XML format and it has been parsed to CSV, `parsed.csv` (through beautifulsoup).

This notebook contains the code for extracting TF.IDF features for the GermEval dataset. 

- Title and Description fields are combined before applying TF.IDF
- TF.IDF is also applied to the Authors field separately. 

Different classifiers are tested in their vanilla form on the TF.IDF alone

TODO:
- Stopwords removal (German)
- Root words extraction (maybe)

In [94]:
import pandas as pd
import pickle
from ast import literal_eval
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, cohen_kappa_score, f1_score
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MultiLabelBinarizer , LabelEncoder , LabelBinarizer
from sklearn.svm import LinearSVC , NuSVC
from utils import subtask_A_evaluation ,subtask_B_evaluation
from bpemb import BPEmb


In [95]:
bpemb_de = BPEmb(lang="de", vs=25000, dim=300)

In [96]:
bpemb_de.encode("Ein Blick hinter die ")

['▁ein', '▁blick', '▁hinter', '▁die']

In [97]:
book_df = pd.read_csv("/home/evenuma/germeval/data/parsed_train_plus_validation.csv")

The categories can be represented as lists wherein each element is a tag. Each tag is separated in to levels split by '>' token. 

In [98]:
book_df["categories"] = book_df["categories"].apply(lambda categories: literal_eval(categories))

In [99]:
def get_mid_level(cat_path):
    cats = cat_path.split(" > ")
    if len(cats) > 1:
        return cats[1]
    else:
        return cats[0]

For the time being, we are focusing on a single tag classification and only the top level is considered. If a sample has more than one label assigned to it, we take only the first.

In [100]:
book_df["top_level"] = book_df["categories"]\
    .apply(lambda  categories: np.unique([get_mid_level(i) for i in categories]))

In [101]:
book_df["count_of_categories"] = book_df["top_level"].apply(lambda top_levels: 3 if len(top_levels) > 3 else len(top_levels))

In [102]:
book_df.count_of_categories.value_counts()

1    12286
2    3133 
3    1208 
Name: count_of_categories, dtype: int64

In [103]:
book_df.columns

Index(['title', 'description', 'categories', 'author', 'published_date',
       'isbn', 'top_level', 'count_of_categories'],
      dtype='object')

In [104]:
pd.set_option('display.max_colwidth', -1)

In [105]:
book_df[["categories","top_level"]].head(3)

,categories,top_level
0,[Literatur & Unterhaltung > Romane & Erzählungen],[Romane & Erzählungen]
1,[Literatur & Unterhaltung > Fantasy > Heroische Fantasy],[Fantasy]
2,[Ratgeber > Lebenshilfe & Psychologie > Besser leben],[Lebenshilfe & Psychologie]


In [106]:
book_df["toplevel_categories"] = book_df["categories"].apply(lambda x:np.unique([i.split(" > ")[0].strip() for i in x]).tolist())

In [107]:
def repeat_categories(df):
    lens = [len(item) for item in df['top_level']]
    return pd.DataFrame({"category" : np.concatenate(df['top_level'].values), 
                         "categories" : np.repeat(df['top_level'].values,lens), 
                          "title" : np.repeat(df['title'].values,lens),
                          "description" : np.repeat(df['description'].values,lens),
                          "author" : np.repeat(df['author'].values,lens),
                          "published_date" : np.repeat(df['published_date'].values,lens),
                          "isbn":np.repeat(df['isbn'].values,lens),
                          "count_of_categories":np.repeat(df['count_of_categories'].values,lens),
                          "toplevel_categories":np.repeat(df['toplevel_categories'].values,lens)})

In [108]:
# len(np.repeat(book_df['categories'].values,lens))

In [109]:
flat_book_df = repeat_categories(book_df)

In [110]:
flat_book_df.head(3)

,category,categories,title,description,author,published_date,isbn,count_of_categories,toplevel_categories
0,Romane & Erzählungen,[Romane & Erzählungen],Die Klinik,"Ein Blick hinter die Kulissen eines Krankenhauses vom Autor der Bestseller ""Der Medicus"" und ""Der Medicus von Saragossa"". Der Wissenschaftler Adam Silverstone, der kubanische Aristokrat Rafael Meomartino und der Farbige Spurgeon Robinson - sie sind drei grundverschiedene Klinik-Ärzte, die unter der unerbittlichen Aufsicht von Dr. Longwood praktizieren. Eines Tages stirbt eine Patientin, und Dr. Longwood wittert einen Behandlungsfehler. Sofort macht er sich auf die Suche nach einem Schuldigen, dem er die Verantwortung in die Schuhe schieben könnte ...",Noah Gordon,2013-12-02,9783641136291,1,[Literatur & Unterhaltung]
1,Fantasy,[Fantasy],Die Erben von Midkemia 4,"Die Bedrohungen für Midkemia und Kelewan wollen nicht enden: Obwohl das Konklave der Schatten Leso Varen und seinen Nachtfalken dicht auf den Fersen ist, schmieden sie weiter ihre finsteren Umsturzpläne gegen das Herrscherhaus von Kesh. Zugleich stellt sich heraus, dass von den mysteriösen Talnoy eine bisher ungekannte Gefahr ausgeht: durch ihre magischen Kräfte können die fürchterlichen Dasati ins Reich Midkemia eindringen und alle ins Unheil stürzen …",Raymond Feist,2016-06-20,9783641185787,1,[Literatur & Unterhaltung]
2,Lebenshilfe & Psychologie,[Lebenshilfe & Psychologie],Völlig losgelöst,"In der Dreizimmerwohnung stapeln sich Flohmarktfunde und modische Fehlkäufe, und durch den lang erhofften Karrieresprung dreht sich das Hamsterrad nur noch schneller. Vielen Menschen geht es ähnlich: In unserer Welt voll Hektik und Überfluss sehnen wir uns danach, uns wieder auf die wirklich wichtigen Dinge zu konzentrieren und Zeit zu haben für Familie, Freunde und uns selbst. Konsumforscher, Psychologen und überzeugte Minimalisten zeigen in diesem Buch, wie wir uns von Unwichtigem trennen, nachhaltig konsumieren und uns endlich wieder frei fühlen – denn weniger ist in der Tat mehr.",Susanne Weingarten,2019-01-14,9783328103646,1,[Ratgeber]


In [111]:
category_value_counts = flat_book_df.category.value_counts()
less_count_categories = category_value_counts[category_value_counts < 4].index.tolist()

In [112]:
less_count_categories

[]

In [113]:
print(len(flat_book_df))
print(sum(book_df["count_of_categories"].values))

22780
22176


## ^ The above diff is due to 4 records having 4 toplevel categories and they have been limited to 3

In [114]:
final_df = flat_book_df[~flat_book_df.category.isin(less_count_categories)]

In [115]:
final_df["authors"] = final_df["author"].apply(lambda x:[ i.strip() for i in str(x).split(",")])

In [116]:
final_df["published_date_parsed"] = pd.to_datetime(final_df["published_date"],infer_datetime_format=True)

In [117]:
final_df["year"] = final_df["published_date_parsed"].apply(lambda x: x.year)

In [118]:
final_df['ISBN_GRP'] = final_df["isbn"].apply(lambda x:str(x)[4:6])
final_df['ISBN_PUBLISHER'] = final_df["isbn"].apply(lambda x:str(x)[6:10]) 

In [119]:
final_df.head(3)

,category,categories,title,description,author,published_date,isbn,count_of_categories,toplevel_categories,authors,published_date_parsed,year,ISBN_GRP,ISBN_PUBLISHER
0,Romane & Erzählungen,[Romane & Erzählungen],Die Klinik,"Ein Blick hinter die Kulissen eines Krankenhauses vom Autor der Bestseller ""Der Medicus"" und ""Der Medicus von Saragossa"". Der Wissenschaftler Adam Silverstone, der kubanische Aristokrat Rafael Meomartino und der Farbige Spurgeon Robinson - sie sind drei grundverschiedene Klinik-Ärzte, die unter der unerbittlichen Aufsicht von Dr. Longwood praktizieren. Eines Tages stirbt eine Patientin, und Dr. Longwood wittert einen Behandlungsfehler. Sofort macht er sich auf die Suche nach einem Schuldigen, dem er die Verantwortung in die Schuhe schieben könnte ...",Noah Gordon,2013-12-02,9783641136291,1,[Literatur & Unterhaltung],[Noah Gordon],2013-12-02,2013,64,1136
1,Fantasy,[Fantasy],Die Erben von Midkemia 4,"Die Bedrohungen für Midkemia und Kelewan wollen nicht enden: Obwohl das Konklave der Schatten Leso Varen und seinen Nachtfalken dicht auf den Fersen ist, schmieden sie weiter ihre finsteren Umsturzpläne gegen das Herrscherhaus von Kesh. Zugleich stellt sich heraus, dass von den mysteriösen Talnoy eine bisher ungekannte Gefahr ausgeht: durch ihre magischen Kräfte können die fürchterlichen Dasati ins Reich Midkemia eindringen und alle ins Unheil stürzen …",Raymond Feist,2016-06-20,9783641185787,1,[Literatur & Unterhaltung],[Raymond Feist],2016-06-20,2016,64,1185
2,Lebenshilfe & Psychologie,[Lebenshilfe & Psychologie],Völlig losgelöst,"In der Dreizimmerwohnung stapeln sich Flohmarktfunde und modische Fehlkäufe, und durch den lang erhofften Karrieresprung dreht sich das Hamsterrad nur noch schneller. Vielen Menschen geht es ähnlich: In unserer Welt voll Hektik und Überfluss sehnen wir uns danach, uns wieder auf die wirklich wichtigen Dinge zu konzentrieren und Zeit zu haben für Familie, Freunde und uns selbst. Konsumforscher, Psychologen und überzeugte Minimalisten zeigen in diesem Buch, wie wir uns von Unwichtigem trennen, nachhaltig konsumieren und uns endlich wieder frei fühlen – denn weniger ist in der Tat mehr.",Susanne Weingarten,2019-01-14,9783328103646,1,[Ratgeber],[Susanne Weingarten],2019-01-14,2019,32,8103


In [120]:
validation_authors =  pickle.load(open("validation_authors",'rb'))
test_authors =  pickle.load(open("test_authors",'rb'))

In [121]:
all_authors = np.concatenate((validation_authors.values,final_df["authors"].values))

In [122]:
mlb = MultiLabelBinarizer()
mlb.fit(all_authors)

year_lb = LabelBinarizer()
year_lb.fit(final_df["year"])

toplevel_category_mlb = MultiLabelBinarizer()
toplevel_category_mlb.fit(final_df["toplevel_categories"].tolist())

isbn_group_lb = LabelBinarizer()
isbn_group_lb.fit(final_df["ISBN_GRP"])

isbn_publisher_lb = LabelBinarizer()
isbn_publisher_lb.fit(final_df["ISBN_PUBLISHER"])

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [123]:
final_df["authors"].values[:2]

array([list(['Noah Gordon']), list(['Raymond Feist'])], dtype=object)

In [124]:
def combine_title_desc(row):
    """Combines the Title and Description fields in the given row
    and returns the combined result"""
    return str(row["title"]) + " " + str(row["description"])
    #return str(row["title"]) + " " + str(row["description"]) + " " + str(row["author"])

In [125]:
final_df.head(3)

,category,categories,title,description,author,published_date,isbn,count_of_categories,toplevel_categories,authors,published_date_parsed,year,ISBN_GRP,ISBN_PUBLISHER
0,Romane & Erzählungen,[Romane & Erzählungen],Die Klinik,"Ein Blick hinter die Kulissen eines Krankenhauses vom Autor der Bestseller ""Der Medicus"" und ""Der Medicus von Saragossa"". Der Wissenschaftler Adam Silverstone, der kubanische Aristokrat Rafael Meomartino und der Farbige Spurgeon Robinson - sie sind drei grundverschiedene Klinik-Ärzte, die unter der unerbittlichen Aufsicht von Dr. Longwood praktizieren. Eines Tages stirbt eine Patientin, und Dr. Longwood wittert einen Behandlungsfehler. Sofort macht er sich auf die Suche nach einem Schuldigen, dem er die Verantwortung in die Schuhe schieben könnte ...",Noah Gordon,2013-12-02,9783641136291,1,[Literatur & Unterhaltung],[Noah Gordon],2013-12-02,2013,64,1136
1,Fantasy,[Fantasy],Die Erben von Midkemia 4,"Die Bedrohungen für Midkemia und Kelewan wollen nicht enden: Obwohl das Konklave der Schatten Leso Varen und seinen Nachtfalken dicht auf den Fersen ist, schmieden sie weiter ihre finsteren Umsturzpläne gegen das Herrscherhaus von Kesh. Zugleich stellt sich heraus, dass von den mysteriösen Talnoy eine bisher ungekannte Gefahr ausgeht: durch ihre magischen Kräfte können die fürchterlichen Dasati ins Reich Midkemia eindringen und alle ins Unheil stürzen …",Raymond Feist,2016-06-20,9783641185787,1,[Literatur & Unterhaltung],[Raymond Feist],2016-06-20,2016,64,1185
2,Lebenshilfe & Psychologie,[Lebenshilfe & Psychologie],Völlig losgelöst,"In der Dreizimmerwohnung stapeln sich Flohmarktfunde und modische Fehlkäufe, und durch den lang erhofften Karrieresprung dreht sich das Hamsterrad nur noch schneller. Vielen Menschen geht es ähnlich: In unserer Welt voll Hektik und Überfluss sehnen wir uns danach, uns wieder auf die wirklich wichtigen Dinge zu konzentrieren und Zeit zu haben für Familie, Freunde und uns selbst. Konsumforscher, Psychologen und überzeugte Minimalisten zeigen in diesem Buch, wie wir uns von Unwichtigem trennen, nachhaltig konsumieren und uns endlich wieder frei fühlen – denn weniger ist in der Tat mehr.",Susanne Weingarten,2019-01-14,9783328103646,1,[Ratgeber],[Susanne Weingarten],2019-01-14,2019,32,8103


In [126]:
final_df.category.value_counts()

Romane & Erzählungen                    2632
Krimi & Thriller                        2313
Frauenunterhaltung                      1303
Science Fiction                         1238
Echtes Leben, Realistischer Roman       1120
Fantasy                                 853 
Liebe, Beziehung und Freundschaft       683 
Politik & Gesellschaft                  675 
Lebenshilfe & Psychologie               603 
Abenteuer                               580 
Biographien & Autobiographien           524 
Fantasy und Science Fiction             450 
Gesundheit & Ernährung                  435 
Historische Romane                      410 
Kinderbuch & Jugendbuch                 388 
Ganzheitliche Psychologie               379 
(Zeit-) Geschichte                      370 
Essen & Trinken                         335 
Spiritualität & Religion                323 
Eltern & Familie                        315 
Psychologie                             308 
Freizeit & Hobby                        262 
Theologie 

### Prepare the data

In [127]:
del book_df
del flat_book_df

import gc
gc.collect()

124

In [128]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
final_df["mid_level"] = final_df["category"].apply(lambda x:get_mid_level(x))
final_df["target"]  = le.fit_transform(final_df["mid_level"].values)

data = train_test_split(final_df[["title","description","authors","category","year","count_of_categories","isbn","categories","toplevel_categories","ISBN_GRP","ISBN_PUBLISHER"]],
                final_df["target"], test_size=0.20, random_state=42, stratify=final_df["target"], shuffle=True)

X_train_df, X_test_df, y_train, y_test = data
vectorizer = TfidfVectorizer(ngram_range=(1,2),stop_words=stopwords.words('german'),tokenizer=bpemb_de.encode)

X_train = vectorizer.fit_transform(
    X_train_df.apply(lambda row: combine_title_desc(row), axis=1))
X_test = vectorizer.transform(X_test_df.apply(lambda row: combine_title_desc(row),axis=1))

X_train_author = mlb.transform(X_train_df["authors"].values)
X_test_author = mlb.transform(X_test_df["authors"].values)

X_train_year = year_lb.transform(X_train_df["year"].values)
X_test_year = year_lb.transform(X_test_df["year"].values)

X_train_isbn_grp = isbn_group_lb.transform(X_train_df["ISBN_GRP"].values)
X_test_isbn_grp = isbn_group_lb.transform(X_test_df["ISBN_GRP"].values)

X_train_isbn_publisher = isbn_publisher_lb.transform(X_train_df["ISBN_PUBLISHER"].values)
X_test_isbn_publisher = isbn_publisher_lb.transform(X_test_df["ISBN_PUBLISHER"].values)


# X_train_top_level_categories = toplevel_category_mlb.transform(X_train_df["toplevel_categories"].values)
# X_test_top_level_categories = toplevel_category_mlb.transform(X_test_df["toplevel_categories"].values)
# X_train_categories = ["_".join([str(le.transform([a])[0]) for a in i]) for i in  X_train_df["categories"]]
# X_test_categories = ["_".join([str(le.transform([a])[0]) for a in i]) for i in  X_test_df["categories"]]

from scipy import sparse
X_train_all = sparse.hstack((
     X_train,sparse.csr_matrix(X_train_author),
     sparse.csr_matrix(X_train_year),
     sparse.csr_matrix(X_train_isbn_grp),
     sparse.csr_matrix(X_train_isbn_publisher),    
#      sparse.csr_matrix(X_train_top_level_categories),
     sparse.csr_matrix(X_train_df["isbn"].values.reshape(len(X_train_df),1)),    
     sparse.csr_matrix(X_train_df["count_of_categories"].values.reshape(len(X_train_df),1))
    ))

X_test_all = sparse.hstack((
    X_test,sparse.csr_matrix(X_test_author),
    sparse.csr_matrix(X_test_year),
     sparse.csr_matrix(X_test_isbn_grp),
     sparse.csr_matrix(X_test_isbn_publisher),    
    
#     sparse.csr_matrix(X_test_top_level_categories),
    sparse.csr_matrix(X_test_df["isbn"].values.reshape(len(X_test_df),1)),
    sparse.csr_matrix(X_test_df["count_of_categories"].values.reshape(len(X_test_df),1))
    ))



/home/evenuma/anaconda3/envs/deeplearning_36/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['be', 'chem', 'ches', 'em', 'en', 'ig', 'igem', 'iges', 'm', 'r', 'sel', 'selben', 'st', 'uer', 'ure', 'uren', '▁aber', '▁alle', '▁allem', '▁allen', '▁aller', '▁alles', '▁als', '▁also', '▁am', '▁an', '▁ander', '▁andere', '▁anderem', '▁anderen', '▁anderer', '▁anderes', '▁andern', '▁anders', '▁auch', '▁auf', '▁aus', '▁bei', '▁bin', '▁bis', '▁bist', '▁d', '▁da', '▁damit', '▁dann', '▁das', '▁dasselbe', '▁dazu', '▁daß', '▁dein', '▁dem', '▁demselben', '▁den', '▁denn', '▁denselben', '▁der', '▁derer', '▁derselben', '▁des', '▁desselben', '▁dessen', '▁dich', '▁die', '▁dies', '▁diese', '▁dieselbe', '▁diesem', '▁diesen', '▁dieser', '▁dieses', '▁dir', '▁doch', '▁dort', '▁du', '▁durch', '▁e', '▁ein', '▁eine', '▁einem', '▁einen', '▁einer', '▁eines', '▁einige', '▁einigen', '

In [129]:
pickle.dump(mlb, open("mlb_ete_top_level.pickle", "wb"))
pickle.dump(year_lb, open("lb_ete_top_level.pickle", "wb"))
pickle.dump(vectorizer, open("vectorizer_ete_top_level.pickle", "wb"))
pickle.dump(X_train_all, open("X_train_ete_top_level.pickle", "wb"))
pickle.dump(X_test_all, open("X_test_ete_top_level.pickle", "wb"))
pickle.dump(y_train, open("y_train_ete_top_level.pickle", "wb"))
pickle.dump(y_test, open("y_test_ete_top_level.pickle", "wb"))
pickle.dump(dict(zip(le.classes_,le.transform(le.classes_))), open("le_author_year_mapping.pickle", "wb"))

In [130]:
%%time
from scipy import sparse

X_full = sparse.vstack((X_train_all,X_test_all))
Y_full = np.concatenate((y_train,y_test))

CPU times: user 89.8 ms, sys: 26.1 ms, total: 116 ms
Wall time: 114 ms


In [131]:
Y_full

array([14, 17, 23, ..., 36, 51, 66])

In [132]:
X_full_array = X_full.tocsr()


In [133]:
# class_weights = {'Kinderbuch & Jugendbuch': 1.8, 'Ratgeber': 3, 'Sachbuch': 2,'Glaube & Ethik' : 2 ,'Künste' : 6,'Architektur & Garten' : 6 }

In [134]:
# class_weights_encoded = []
# for value,key in enumerate(class_weights):
#     print(class_weights[key],key)
#     class_weights_encoded.append((le.transform([key])[0],class_weights[key]))

In [135]:
# dict(class_weights_encoded)
class_weights_encoded = {}

In [136]:
global_isbn_category_dict = [(i[0],[le.transform([k])[0] for k in i[1].tolist()]) for i in final_df[["isbn","categories"]].head(100).values]

global_isbn_category_df = pd.DataFrame(global_isbn_category_dict,columns=["isbn","actual"])

global_isbn_category_df["isbn"] = global_isbn_category_df["isbn"].apply(lambda x:str(x))

def get_one_instance(index,csr_matrix):
    unique_isbns = set([ str(int(i[0])) for i in csr_matrix[index,-2].todense()])
    isbns_dict = dict(list(zip([ str(int(i[0])) for i in csr_matrix[index,-2].todense()],index)))
    return (list(isbns_dict.keys()),list(isbns_dict.values()))

def getClasses(class_probs,class_count):
    arr = np.array(class_probs)
    return arr.argsort()[-1 * class_count:][::-1]

def get_multi_prediction(row):
    """Combines the Title and Description fields in the given row
    and returns the combined result"""
    return getClasses(row["predict_proba"],row["class_count"])

def compute_f1_score(prediction,class_count_prediction):
    prediction_df = pd.DataFrame(prediction,columns=["isbn","predict_proba"])
    prediction_df["isbn"] = prediction_df["isbn"].apply(lambda x:str(int(x[0])))
#     print(prediction_df)
    class_count_df = pd.DataFrame(class_count_prediction,columns=["isbn","class_count"])
    
    joined_ = prediction_df.merge(class_count_df,on="isbn",how="inner")
    
    joined_["predictions"] = joined_.apply(get_multi_prediction,axis=1)
    joined_1 = joined_.merge(global_isbn_category_df,on="isbn")
    predicted_ = joined_1["predictions"]
    actuals_ = joined_1["actual"]
    
    predicted_labels = [ [le.inverse_transform([k])[0] for k in i] for i in predicted_.values]
    actual_labels = [ [le.inverse_transform([k])[0] for k in i] for i in actuals_.values]
    
    return subtask_B_evaluation(actual_labels,predicted_labels)
    
# compute_f1_score(a,b)

# Build End to end classifer

In [137]:
a = None
b = None
from sklearn.model_selection import StratifiedKFold
fold_classes = []
probas = []
cv = StratifiedKFold(n_splits=4, random_state=42)
for train_index, test_index in cv.split(X_full_array,Y_full):      
#     print("Train Index: ", train_index, "\n")
#     print("Test Index: ", len(test_index))
    
    lsvcclf = LinearSVC(random_state=42,max_iter=3000,verbose=1,C=0.5)
    X_train, X_test, y_train, y_test = X_full_array[train_index,:-2], X_full_array[test_index,:-2], Y_full[train_index], Y_full[test_index]

    lsvcclf.fit(X_train,y_train)
    
    print("printing prediction")
    prediction = list(zip(X_full_array[test_index,-2].todense().tolist(),lsvcclf._predict_proba_lr(X_test)))
    a = prediction
#     print(prediction)

    train_isbn , train_index_for_cc = get_one_instance(train_index,X_full_array)
    test_isbn , test_index_for_cc = get_one_instance(test_index,X_full_array)
    
    X_train_cc, X_test_cc = X_full_array[train_index_for_cc,:-2], X_full_array[test_index_for_cc,:-2]
    
    y_train_cc, y_test_cc = [int(i[0]) for i in X_full_array[train_index_for_cc,-1].todense().tolist()], [int(i[0]) for i in  X_full_array[test_index_for_cc,-1].todense().tolist()]

    lsvcclf_cc = LinearSVC(random_state=42,max_iter=3000,verbose=1)
    
    lsvcclf_cc.fit(X_train_cc,y_train_cc)
    
    prediction_cc = list(zip(test_isbn,lsvcclf_cc.predict(X_test_cc)))
    b = prediction_cc    
    
    print(compute_f1_score(prediction,prediction_cc))

    

[LibLinear]printing prediction
[LibLinear][0.6470588235294118, 0.9041095890410958, 0.7542857142857143, 0.35294117647058826]
[LibLinear]printing prediction
[LibLinear][0.7, 0.8032786885245902, 0.7480916030534351, 0.4722222222222222]
[LibLinear]printing prediction
[LibLinear][0.7368421052631579, 0.7466666666666667, 0.7417218543046358, 0.6136363636363636]
[LibLinear]printing prediction
[LibLinear][0.6865671641791045, 0.8932038834951457, 0.7763713080168777, 0.36538461538461536]


In [138]:
X_full_array.shape

(22780, 643684)

In [139]:
le.transform(le.classes_)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94])

## Test Data Creation

In [140]:
test_book_df = pd.read_csv("/home/evenuma/germeval/data/test_set.csv")

In [141]:
test_book_df.head(3)

,title,description,author,published_date,isbn
0,Malbuch für 365 Tage,"Ausmalen bringt Freude und entspannt. Dieses meditative Ausmalbuch begleitet Sie mit seinen vielfältigen und wunderschönen Motiven sogar durch das ganze Jahr. Jedem Motiv ist eine inspirierende Affirmation zugeordnet, nach der Sie Ihren Alltag gestalten können. Zudem reduziert tägliches Ausmalen den Stress, erhöht Ihr Wohlbefinden, fördert die Kreativität, verbessert Ihre Konzentration und ist Balsam für die Seelen.",NaN,2016-10-03,9783809436690
1,Ansteckende Gesundheit,"Die Beliebtheit der Geistheilung als Alternative zur Schulmedizin wächst ständig. Aber warum ist Geistheilung so erfolgreich? Einer der renommiertesten Heiler Europas gibt unmittelbare Einblicke in seine Tätigkeit. Seriös, kompetent und verständlich beantwortet Horst Krohne die grundlegenden Fragen, stellt Diagnose- und Behandlungsmöglichkeiten vor und zeigt Chancen und Grenzen auf. Für alle, die sich umfassend über Geistheilung informieren oder sich selbst behandeln lassen wollen.",Horst Krohne,2014-11-10,9783453702653
2,Karibu heißt willkommen,"Die englische Farmerstochter Stella und das Kikuyu-Mädchen Lilly wachsen auf einer Farm im Hochland Kenias wie Schwestern auf. Als der Kampf um Kenias Unabhängigkeit auch die Farm Karibu erreicht, muss Stella nach London gehen. Sie beginnt ein neues Leben – bis zu dem Augenblick, als die unstillbare Sehnsucht nach Afrika sie wieder einholt.",Stefanie Zweig,2010-04-06,9783453407343


In [142]:
test_book_df["authors"] = test_book_df["author"].apply(lambda x:[ i.strip() for i in str(x).split(",")])

test_book_df["published_date_parsed"] = pd.to_datetime(final_df["published_date"],infer_datetime_format=True)

test_book_df["year"] = final_df["published_date_parsed"].apply(lambda x: x.year)

In [143]:
test_book_df['ISBN_GRP'] = test_book_df["isbn"].apply(lambda x:str(x)[4:6])
test_book_df['ISBN_PUBLISHER'] = test_book_df["isbn"].apply(lambda x:str(x)[6:10]) 

In [144]:
test_book_df.head(3)

,title,description,author,published_date,isbn,authors,published_date_parsed,year,ISBN_GRP,ISBN_PUBLISHER
0,Malbuch für 365 Tage,"Ausmalen bringt Freude und entspannt. Dieses meditative Ausmalbuch begleitet Sie mit seinen vielfältigen und wunderschönen Motiven sogar durch das ganze Jahr. Jedem Motiv ist eine inspirierende Affirmation zugeordnet, nach der Sie Ihren Alltag gestalten können. Zudem reduziert tägliches Ausmalen den Stress, erhöht Ihr Wohlbefinden, fördert die Kreativität, verbessert Ihre Konzentration und ist Balsam für die Seelen.",NaN,2016-10-03,9783809436690,[nan],2013-12-02,2013,80,9436
1,Ansteckende Gesundheit,"Die Beliebtheit der Geistheilung als Alternative zur Schulmedizin wächst ständig. Aber warum ist Geistheilung so erfolgreich? Einer der renommiertesten Heiler Europas gibt unmittelbare Einblicke in seine Tätigkeit. Seriös, kompetent und verständlich beantwortet Horst Krohne die grundlegenden Fragen, stellt Diagnose- und Behandlungsmöglichkeiten vor und zeigt Chancen und Grenzen auf. Für alle, die sich umfassend über Geistheilung informieren oder sich selbst behandeln lassen wollen.",Horst Krohne,2014-11-10,9783453702653,[Horst Krohne],2016-06-20,2016,45,3702
2,Karibu heißt willkommen,"Die englische Farmerstochter Stella und das Kikuyu-Mädchen Lilly wachsen auf einer Farm im Hochland Kenias wie Schwestern auf. Als der Kampf um Kenias Unabhängigkeit auch die Farm Karibu erreicht, muss Stella nach London gehen. Sie beginnt ein neues Leben – bis zu dem Augenblick, als die unstillbare Sehnsucht nach Afrika sie wieder einholt.",Stefanie Zweig,2010-04-06,9783453407343,[Stefanie Zweig],2019-01-14,2019,45,3407


In [145]:
from sklearn import preprocessing
# data = train_test_split(final_df[["title","description","authors","category","year","count_of_categories","isbn","categories"]],
#                 final_df["target"], test_size=0.20, random_state=42, stratify=final_df["target"], shuffle=True)

# X_train_df, X_test_df, y_train, y_test = data
# vectorizer = TfidfVectorizer(ngram_range=(1,2),stop_words=stopwords.words('german'),tokenizer=bpemb_tokenize)

X_test_set = vectorizer.transform(test_book_df.apply(lambda row: combine_title_desc(row), axis=1))

X_test_set_author = mlb.transform(test_book_df["authors"].values)

X_test_set_year = year_lb.transform(test_book_df["year"].values)

X_test_ISBN_GRP = isbn_group_lb.transform(test_book_df["ISBN_GRP"].values)

X_test_ISBN_PUBLISHER = isbn_publisher_lb.transform(test_book_df["ISBN_PUBLISHER"].values)

# X_train_categories = ["_".join([str(le.transform([a])[0]) for a in i]) for i in  X_train_df["categories"]]
# X_test_categories = ["_".join([str(le.transform([a])[0]) for a in i]) for i in  X_test_df["categories"]]

from scipy import sparse
X_test_set_all = sparse.hstack((
     X_test_set,sparse.csr_matrix(X_test_set_author),
     sparse.csr_matrix(X_test_set_year),
     sparse.csr_matrix(X_test_ISBN_GRP),
     sparse.csr_matrix(X_test_ISBN_PUBLISHER)
    ))

# X_test_all = sparse.hstack((
#     X_test,sparse.csr_matrix(X_test_author),
#     sparse.csr_matrix(X_test_year),
#     sparse.csr_matrix(X_test_df["isbn"].values.reshape(len(X_test_df),1)),
#     sparse.csr_matrix(X_test_df["count_of_categories"].values.reshape(len(X_test_df),1))
#     ))



/home/evenuma/anaconda3/envs/deeplearning_36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:951: UserWarning: unknown class(es) ['A. D. Jameson', 'A3R Roberts', 'Aaravindha Himadra', 'Abaelard', 'Abby McDonald', 'Abdullah Takim', 'Abhijit Banerjee', 'Abraham Verghese', 'Achim Lucchesi', 'Achim Plagentz', 'Achim Wohlgethan', 'Adam Leonard', 'Adelia Saunders', 'Aeneas Rooch', 'Ahlea Khadro', 'Ahmet Ümit', 'Akila Berjaoui', 'Alan Alexander Milne', 'Alan Burdick', 'Albrecht Grözinger', 'Alden Bell', 'Aldo Keel', 'Alessandro Biamonti', 'Alexander Langer', 'Alexander Monro', 'Alexander Poraj', 'Alexander Steffensmeier', 'Alexander Vance', 'Alexandra Borchardt', 'Alexandra Ehlert', 'Alexandra Frank', 'Alexandra Fuller', 'Alexis Morgan', 'Alfred Brendel', 'Alfred Dorfer', 'Alfred Lansing', 'Alfred Schindler', 'Ali Land', 'Alice Curtis', 'Alice Peuple', 'Alma Katsu', 'Amber Hatch', 'Ami McKay', 'Amir Levine', 'Amity Gaige', 'Amy Mullins', 'An Kuei Chi', 'Ana Lilia Pérez', 'Andrea Br

In [146]:
X_full_array_arr = X_full_array[:,:-2].toarray()

## Create Final Model 

In [147]:
%%time
lsvcclf = LinearSVC(random_state=42,max_iter=3000,verbose=1,class_weight= dict(class_weights_encoded))
lsvcclf.fit(X_full_array_arr,Y_full)

[LibLinear]CPU times: user 2min 55s, sys: 517 ms, total: 2min 56s
Wall time: 2min 56s


In [148]:
%%time
lsvcclf_cc = LinearSVC(random_state=42,max_iter=3000,verbose=1)
lsvcclf_cc.fit(X_full_array_arr,[int(i[0]) for i in X_full_array[:,-1].todense().tolist()])

[LibLinear]CPU times: user 45.2 s, sys: 781 ms, total: 46 s
Wall time: 45.9 s


In [149]:
def getClasses(class_probs,class_count):
    arr = np.array(class_probs)
    return arr.argsort()[-1 * class_count:][::-1]

## Predict for the test set

In [150]:
test_book_df["class_count"] = lsvcclf_cc.predict(X_test_set_all)

In [151]:
y_prob_list = [i.tolist() for i in lsvcclf._predict_proba_lr(X_test_set_all)]

In [152]:
test_book_df["class_probs"] = y_prob_list

In [153]:
test_book_df.head(2)

,title,description,author,published_date,isbn,authors,published_date_parsed,year,ISBN_GRP,ISBN_PUBLISHER,class_count,class_probs
0,Malbuch für 365 Tage,"Ausmalen bringt Freude und entspannt. Dieses meditative Ausmalbuch begleitet Sie mit seinen vielfältigen und wunderschönen Motiven sogar durch das ganze Jahr. Jedem Motiv ist eine inspirierende Affirmation zugeordnet, nach der Sie Ihren Alltag gestalten können. Zudem reduziert tägliches Ausmalen den Stress, erhöht Ihr Wohlbefinden, fördert die Kreativität, verbessert Ihre Konzentration und ist Balsam für die Seelen.",NaN,2016-10-03,9783809436690,[nan],2013-12-02,2013,80,9436,1,"[0.010351228047532645, 0.011135365746542883, 0.009509887862025055, 0.008093318276156244, 0.01040599794288465, 0.013370483928009791, 0.011511408168829056, 0.01008660113344966, 0.010075960455905718, 0.00921133414144666, 0.008865237966714585, 0.009123106145043228, 0.012056485786354411, 0.011497965662510148, 0.011485767444641764, 0.011552332949821243, 0.011201931001414974, 0.014749185316227301, 0.010494412345653565, 0.008940632681269453, 0.011038442299557024, 0.010839367951299236, 0.009573178574743236, 0.008756315085080107, 0.021586629683406286, 0.011809981341596165, 0.011173526139023487, 0.008763046249433344, 0.01138786344292559, 0.010807794895255914, 0.011206271177833547, 0.010981835610890934, 0.011888313076695379, 0.01072993016014388, 0.009486897934462378, 0.009972808817601302, 0.010065464453754688, 0.010447490576281257, 0.011757114989066321, 0.00823220566450409, 0.008989945385705795, 0.009623578515982546, 0.013613032537542053, 0.011369390451242512, 0.008054212132263098, 0.008355684117332425, 0.011613924340101818, 0.011380215320855425, 0.009633490580328278, 0.01681466348073924, 0.01059688711523932, 0.007864830220485106, 0.00826685496427351, 0.009159301481942703, 0.010712847733761775, 0.010258747075119522, 0.009365620458191221, 0.01086356807536109, 0.011281906359777367, 0.011324246201963697, 0.011478462313636015, 0.010316736636312577, 0.011069273162702713, 0.010652808989285419, 0.011032044205501322, 0.009932706596601367, 0.007589663073810685, 0.009132486908896354, 0.011455245388377128, 0.008648487974661848, 0.010668727826676237, 0.0112106118020608, 0.010552562187783695, 0.010905339675105302, 0.011038789140857921, 0.010694092862436935, 0.009972696421079895, 0.010851382552397511, 0.010974501943027123, 0.011039059617478008, 0.010784961910513185, 0.01113966882496096, 0.011299725659102242, 0.008847835797364501, 0.009195988267871932, 0.011400333850292503, 0.008928444167224872, 0.010968018921302228, 0.007529533353973556, 0.009410897633053302, 0.01159378926529046, 0.00981828205398664, 0.010478951411647237, 0.007979594915378172, 0.00804222701515371]"
1,Ansteckende Gesundheit,"Die Beliebtheit der Geistheilung als Alternative zur Schulmedizin wächst ständig. Aber warum ist Geistheilung so erfolgreich? Einer der renommiertesten Heiler Europas gibt unmittelbare Einblicke in seine Tätigkeit. Seriös, kompetent und verständlich beantwortet Horst Krohne die grundlegenden Fragen, stellt Diagnose- und Behandlungsmöglichkeiten vor und zeigt Chancen und Grenzen auf. Für alle, die sich umfassend über Geistheilung informieren oder sich selbst behandeln lassen wollen.",Horst Krohne,2014-11-10,9783453702653,[Horst Krohne],2016-06-20,2016,45,3702,1,"[0.01055331562122876, 0.009945889619702641, 0.010920507848812471, 0.01031765784064765, 0.010736079485810525, 0.01049732287371996, 0.011008382370012861, 0.010062792814495677, 0.010167690268547486, 0.010580771124963482, 0.010055156773428513, 0.010162235811694308, 0.009208622639599186, 0.011339259856972506, 0.022682244315387567, 0.011093460075418712, 0.010241603828728204, 0.010321451843850755, 0.010349311959735328, 0.00912543655681352, 0.010618090045526394, 0.01030546805856155, 0.010553054796897067, 0.009031009505301286, 0.012002467638120571, 0.01148766900337289, 0.009500378267950766, 0.009696969560404034, 0.010500211377618631, 0.010661730672812293, 0.009092618316827483, 0.011044410926999

In [154]:
dir(le)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 'classes_',
 'fit',
 'fit_transform',
 'get_params',
 'inverse_transform',
 'set_params',
 'transform']

In [155]:
le_mapping= dict(zip(le.classes_,le.transform(le.classes_)))
le_mapping_inv = {v: k for k, v in le_mapping.items()}

def reverseMap(number):
    return le_mapping_inv[number]


In [156]:
test_book_df["class_predictions_e"] = test_book_df.apply(lambda x:getClasses(x["class_probs"],x["class_count"]),axis=1)


In [157]:
test_book_df["class_predictions"] = test_book_df["class_predictions_e"].apply(lambda x: [reverseMap(i) for i in x])

In [158]:
test_book_df.head(3)

,title,description,author,published_date,isbn,authors,published_date_parsed,year,ISBN_GRP,ISBN_PUBLISHER,class_count,class_probs,class_predictions_e,class_predictions
0,Malbuch für 365 Tage,"Ausmalen bringt Freude und entspannt. Dieses meditative Ausmalbuch begleitet Sie mit seinen vielfältigen und wunderschönen Motiven sogar durch das ganze Jahr. Jedem Motiv ist eine inspirierende Affirmation zugeordnet, nach der Sie Ihren Alltag gestalten können. Zudem reduziert tägliches Ausmalen den Stress, erhöht Ihr Wohlbefinden, fördert die Kreativität, verbessert Ihre Konzentration und ist Balsam für die Seelen.",NaN,2016-10-03,9783809436690,[nan],2013-12-02,2013,80,9436,1,"[0.010351228047532645, 0.011135365746542883, 0.009509887862025055, 0.008093318276156244, 0.01040599794288465, 0.013370483928009791, 0.011511408168829056, 0.01008660113344966, 0.010075960455905718, 0.00921133414144666, 0.008865237966714585, 0.009123106145043228, 0.012056485786354411, 0.011497965662510148, 0.011485767444641764, 0.011552332949821243, 0.011201931001414974, 0.014749185316227301, 0.010494412345653565, 0.008940632681269453, 0.011038442299557024, 0.010839367951299236, 0.009573178574743236, 0.008756315085080107, 0.021586629683406286, 0.011809981341596165, 0.011173526139023487, 0.008763046249433344, 0.01138786344292559, 0.010807794895255914, 0.011206271177833547, 0.010981835610890934, 0.011888313076695379, 0.01072993016014388, 0.009486897934462378, 0.009972808817601302, 0.010065464453754688, 0.010447490576281257, 0.011757114989066321, 0.00823220566450409, 0.008989945385705795, 0.009623578515982546, 0.013613032537542053, 0.011369390451242512, 0.008054212132263098, 0.008355684117332425, 0.011613924340101818, 0.011380215320855425, 0.009633490580328278, 0.01681466348073924, 0.01059688711523932, 0.007864830220485106, 0.00826685496427351, 0.009159301481942703, 0.010712847733761775, 0.010258747075119522, 0.009365620458191221, 0.01086356807536109, 0.011281906359777367, 0.011324246201963697, 0.011478462313636015, 0.010316736636312577, 0.011069273162702713, 0.010652808989285419, 0.011032044205501322, 0.009932706596601367, 0.007589663073810685, 0.009132486908896354, 0.011455245388377128, 0.008648487974661848, 0.010668727826676237, 0.0112106118020608, 0.010552562187783695, 0.010905339675105302, 0.011038789140857921, 0.010694092862436935, 0.009972696421079895, 0.010851382552397511, 0.010974501943027123, 0.011039059617478008, 0.010784961910513185, 0.01113966882496096, 0.011299725659102242, 0.008847835797364501, 0.009195988267871932, 0.011400333850292503, 0.008928444167224872, 0.010968018921302228, 0.007529533353973556, 0.009410897633053302, 0.01159378926529046, 0.00981828205398664, 0.010478951411647237, 0.007979594915378172, 0.00804222701515371]",[24],[Freizeit & Hobby]
1,Ansteckende Gesundheit,"Die Beliebtheit der Geistheilung als Alternative zur Schulmedizin wächst ständig. Aber warum ist Geistheilung so erfolgreich? Einer der renommiertesten Heiler Europas gibt unmittelbare Einblicke in seine Tätigkeit. Seriös, kompetent und verständlich beantwortet Horst Krohne die grundlegenden Fragen, stellt Diagnose- und Behandlungsmöglichkeiten vor und zeigt Chancen und Grenzen auf. Für alle, die sich umfassend über Geistheilung informieren oder sich selbst behandeln lassen wollen.",Horst Krohne,2014-11-10,9783453702653,[Horst Krohne],2016-06-20,2016,45,3702,1,"[0.01055331562122876, 0.009945889619702641, 0.010920507848812471, 0.01031765784064765, 0.010736079485810525, 0.01049732287371996, 0.011008382370012861, 0.010062792814495677, 0.010167690268547486, 0.010580771124963482, 0.010055156773428513, 0.010162235811694308, 0.009208622639599186, 0.011339259856972506, 0.022682244315387567, 0.011093460075418712, 0.010241603828728204, 0.010321451843850755, 0.010349311959735328, 0.00912543655681352, 0.010618090045526394, 0.01030546805856155, 0.010553054796897067, 0.009031009505301286, 0.012002467638120571, 0.01148766900337289, 0.009500378267950766, 0.009696969560404034, 0.010500211377618631,

In [159]:
def formatOutput(isbn,class_predictions):
    classes = ["\t"+str(i) for i in class_predictions]
    return str(isbn) + "".join(classes) + "\n"

In [160]:
test_book_df[["isbn","class_predictions"]].to_csv("../../data/svm_test_set_submission_sub_task_b.csv",encoding="utf-8",index=False)

In [161]:
test_book_df = pd.read_csv("../../data/svm_test_set_submission_sub_task_b.csv",encoding="utf-8")

In [162]:
test_book_df.head(3)

,isbn,class_predictions
0,9783809436690,['Freizeit & Hobby']
1,9783453702653,['Energieheilung']
2,9783453407343,['Frauenunterhaltung']


In [163]:
test_book_df["class_predictions"] = test_book_df["class_predictions"].apply(lambda x: literal_eval(x))

In [164]:
test_book_df.head(3)

,isbn,class_predictions
0,9783809436690,[Freizeit & Hobby]
1,9783453702653,[Energieheilung]
2,9783453407343,[Frauenunterhaltung]


In [165]:
submissions = test_book_df.apply(lambda x:formatOutput(x["isbn"],x["class_predictions"]),axis=1).values

In [166]:
submissions

array(['9783809436690\tFreizeit & Hobby\n',
       '9783453702653\tEnergieheilung\n',
       '9783453407343\tFrauenunterhaltung\n', ...,
       '9783442715206\tRomane & Erzählungen\n',
       '9783809027003\tKrimi & Thriller\n',
       '9783641074029\tBiographien & Autobiographien\n'], dtype=object)

In [167]:
len(submissions)

4157

In [168]:
submissions = np.insert(submissions,0,"subtask_b\n")

In [169]:
submissions

array(['subtask_b\n', '9783809436690\tFreizeit & Hobby\n',
       '9783453702653\tEnergieheilung\n', ...,
       '9783442715206\tRomane & Erzählungen\n',
       '9783809027003\tKrimi & Thriller\n',
       '9783641074029\tBiographien & Autobiographien\n'], dtype=object)

In [170]:
submission_format = "".join(submissions)

In [171]:
submission_format

'subtask_b\n9783809436690\tFreizeit & Hobby\n9783453702653\tEnergieheilung\n9783453407343\tFrauenunterhaltung\n9783442742394\tRomane & Erzählungen\n9783734104275\tFrauenunterhaltung\n9783641143633\tKrimi & Thriller\n9783641035471\tHistorische Romane\n9783442268474\tFantasy\n9783424631388\tGesundheit & Ernährung\n9783734104718\tKrimi & Thriller\n9783844509571\tKrimi & Thriller\n9783442393329\tLebenshilfe & Psychologie\n9783466372096\tSpiritualität & Religion\n9783894809690\tErotik\n9783641092337\tKrimi & Thriller\tEchtes Leben, Realistischer Roman\n9783442267811\tFantasy\n9783641058142\tPolitik & Gesellschaft\n9783837109290\tKrimi & Thriller\n9783641079321\tFantasy\n9783442382200\tFrauenunterhaltung\n9783734101878\tKrimi & Thriller\n9783641121846\tEchtes Leben, Realistischer Roman\n9783570222393\tFantasy und Science Fiction\tAbenteuer\tEchtes Leben, Realistischer Roman\n9783641011857\tBiographien & Autobiographien\n9783442714544\tKrimi & Thriller\n9783453524224\tFantasy\n9783453085206\t

In [172]:
submission_file = open("../../data/svm_test_set_submission_sub_task_b.txt",'w',encoding="utf-8")

In [173]:
submission_file.writelines(submission_format)

In [174]:
submission_file.close()

In [154]:
len(np.unique(test_book_df.isbn))

4157